<img src="DAMASK_banner.png">

# <font color=red>CONTENTS</font>
1. ## Importing necessary modules [Go There](#imports) <br>
2. ## Section - 01: Cleaning .ang files [Go There](#cleaning)<br>
    1. ### Converting .ang to dream3d files [Go There](#convert_ang)<br>
    2. ### Section - 01_2 - Convert .ctf files to dream3d [Go There](#convert_ctf)<br>
3. ## Section - 02: Generating material.yaml from dream3d file [Go There](#dream3d)<br>
    1. ### Section - 02.1 For single phase material [Go There](#dream3d_single_phase)<br>
4. ## Section - 03: Generating grid file (.vti) from dream3d file [Go There](#grid)<br>
5. ## Section - 04: Creating loadfile (.yaml) [Go There](#loadfile)<br>
    1. ### Section - 04.1: For tensile tests [Go There](#tensile)<br>
    2. ### Section - 04.2: For dwell fatigue tests [Go There](#dwell)<br>
    3. ### Section - 04.3: For normal fatigue tests [Go There](#fatigue)<br>

<a id="imports"></a>
# <font color=blue> <center>Importing Necessary Modules</center> </font>

In [ ]:
import numpy as np
import damask
from damask import ConfigMaterial as cm

***
<br><br>
# <center><font color=blue> **SECTION - 01:** Cleaning .ang files </font><center>

- ### <font color=blue>That is changing values outside the range of [0, 2π] to be inside</font>
- ### <font color=scarlet> Assumes that euler angles in the ang file are in radians. </font>

<font color=red>NOTE: This needs to be done, because .ang files as created may contain some values greater than [0, 2π], which while will be processed by DREAM3D, but during creation of material file, damask will raise error.</font>

In [ ]:
GENERATED_ANG_FILE: str = "YOUR_CREATED_ANG_FILE_FILEPATH"
# Obtained ang file path and filename, from tsl or ebsd or elsewhere

CLEANED_ANG_FILE: str = "YOUR_OUTPUT_FILE_FILEPATH"
# What you want cleaned ang files name to be and where you need it placed
HEADER_LINES: int = "HEADER_LINES_IN_ANG_FILE"
# Lines starting with # (i.e. before data of ebsd starts)

### <font color=grey>Creating few helper functions</font>

In [ ]:
def outside_range(euler_angle: str, range_min: float = 0, range_max: float = 2 * np.pi) -> bool:
    """Takes in euler angle value (as a string), outputs whether they are outside the accepted range of [0, 2pi],
        Returns True if angle is outside the accepted range,
        False otherwise."""
    angle_radians = float(euler_angle)
    return not (range_min <= angle_radians < range_max)

def convert_in_range(euler_angle: str, range_min: float = 0, range_max: float = 2 * np.pi) -> str:
    """Takes in euler angle (as a string), outputs a value by subtracting or adding 2pi values a i in range value."""
    while outside_range(euler_angle, range_min, range_max):
        angle_radians = float(euler_angle)
        if float(euler_angle) < range_min:
            euler_angle = f"{angle_radians + range_max:.5f}"
        else:
            euler_angle = f"{angle_radians - range_max:.5f}"
    return euler_angle

### <font color=red>Taking in data from GENERATE_ANG_FILE and cleaning it </font>

In [ ]:
header = []
data_lines = []
with open(GENERATED_ANG_FILE, "r") as file_read:
    all_data_lines = file_read.readlines()
    for header_lines in all_data_lines[:HEADER_LINES]:
        # Taking in all header lines in the header list.
        header.append(header_lines)
        
    for data_line in all_data_lines[HEADER_LINES:]:
        # Going through all data_line, first convert data_line in a list
        data_line = data_line.split()
        data_line[0] = convert_in_range(data_line[0])
        data_line[1] = convert_in_range(data_line[1], 0, np.pi)
        data_line[2] = convert_in_range(data_line[2])
        data_lines.append(data_line)

### <font color=green>Exporting clean data as CLEANED_ANG_FILE</font>

In [ ]:
# Exporting data 
with open(CLEANED_ANG_FILE, "w") as file_write:
    for header_line in header:
        file_write.write(header_line)
    for datas in data_lines:
        line = ""
        for data in datas:
            line += data
            line += "\t"
        file_write.write(line + "\n")

## <font color=blue>Converting CLEANED_ANG_FILE to dream3d file</font>

### To convert .ang type files to dream3d file, you can use following DREAM3D pipeline
- #### <a href="DREAM3D_Pipelines/AngTodream3d.json">Pipeline to convert .ang file to dream3d file<a> <br>
    This is placed in DREAM3D_Pipelines folder with filename, "AngTodream3d.json"

### <font color=red> Sometime DREAM3D may throw following error when using above pipeline</font>
#### &emsp; &emsp;<font color=red>import EDAX EBSD data(.ang) </font> &emsp; &emsp; END of ang file reached before all data was parsed.
### <font color=green> Solution: <br> &emsp; &emsp; In this case Go to NROWS line of the header, and decrease the count by 1, that should resolve the issue.</font>

# <font color=blue>SECTION - 01_2 : Converting ctf files to dream3d files</font>

### Converting to dream3d file is quite easy, cleaning and others are not required, the pipeline to use is:
- #### <a href="DREAM3D_Pipelines/ctf_to_dream3d.json">Pipeline to convert .ctf file to .dream3d file</a> <br>
This is placed in DREAM3D_Pieplines folder with filename "ctf_to_dream3d.json"

***
# <font color=scarlet><center>SECTION - 01 : END </center></font>
***

<br> <br> <br>

# <font color=blue><center> SECTION-02: Generating material.yaml from dream3d file </center></font> <br>
- ### <font color=blue>Creating a material.yaml file using dream3d file and available material data, (collected from literature or caluclated)</font>
#### <font color=red>**NOTE**: material.yaml file contains material orientation and properties and a must for damask simulation

## <center><font color=purple> SECTION-02.1 Creating material file for single phase material </font><center>

In [ ]:
DREAM_3D_FILE: str = "YOUR_DREAM3D_FILEPATH"
# Path to dream3d file created from methods mentioned above
MATERIAL_CONTANTS_FILE: str = "MATERIAL_CONSTANTS_FILENAME"
# yaml file where you have saved material data needed for simulation, examples in damask GitHub pages

In [ ]:
# creating a config_material object from dream 3d file
config_material = cm.load_DREAM3D(DREAM_3D_FILE)

# Defining homogenization as type pass, if you are using ebsd data from SEM it should be type pass only
config_material['homogenization']['direct']['mechanical'] = {'type': 'pass'}

<br>

## <font color=grey><center> OPTIONAL PROCESSING - START </center></font>
### Optional Processings, Following cells are not necessary, they are optional
- #### Change phase name
- #### [CLICK HERE](#singlephase_rename) to go to the cell to customize name changes

In [ ]:
# Looking through the config_material object for phase names
config_material

<a id="singlephase_rename"></a>
### <font color=red><center> Change following cell to customize renaming of phase. </center></font>

In [ ]:
OLD_PHASE_NAME: str = "OLD_PHASE_NAME_IN_ConfigMaterial_OBJECT"
NEW_PHASE_NAME: str = "WHAT_YOU_WANT_NEW_NAME_TO_BE"

In [ ]:
# Deleting old phase in phase section
del (config_material['phase'][OLD_PHASE_NAME])
# Adding new phase in phase section
config_material['phase'][NEW_PHASE_NAME] = None

# Renaming phase in material section
if isinstance(config_material['material'][0]['constituents'][0]['phase'], int):
    config_material = config_material.material_rename_phase({int(OLD_PHASE_NAME): NEW_PHASE_NAME})
else:
    config_material = config_material.material_rename_phase({OLD_PHASE_NAME: NEW_PHASE_NAME})


## <font color=grey><center> OPTIONAL PROCESSING - END </center></font>
<br>

### <font color=cadetblue><center> **Adding Material Constants, elastic and plastic to ConfigMaterial Object** </center></font>

In [ ]:
phase_name = list(config_material['phase'].keys())[0]
config_material['phase'][phase_name] = damask.ConfigMaterial.load(MATERIAL_CONTANTS_FILE)

## <font color=cadetblue><center> Saving ConfigMaterial Object (Will be saved as <font color=red>_material.yaml_ </font>)</center></font>

In [ ]:
config_material.save()

## <center><font color=purple> SECTION-02.1 - END </font></center>
<hr>

<hr>

# <font color=blue><center> SECTION-02: END </center></font>
<hr>
<br><br>

<br> <br>

# <font color=blue><center> SECTION-03: Generating grid file from .dream3d file </center></font>
#### <font color=red> &emsp; **NOTE**: grid file is a must for DAMASK_simulation, in simple words it contains placement of voxels (data points) in x,y,z coordinates 

## <font color=cadetblue> <center>Customize name of the grid file <center></font>
- #### <font color=red> &emsp; &emsp; One of two names which will determine result file name.</font>
- #### <font color=red> &emsp; &emsp; Extension of grid file must be .vti </font>

In [ ]:
OUTPUT_GRID_FILE: str = "WHAT_YOU_WANT_GRID_FILENAME_TO_BE"
DREAM_3D_FILE: str = "DREAM3D_FILE_USED_TO_CREATE_MATERIAL_FILE"

In [ ]:
grid_file = damask.Grid.load_DREAM3D(DREAM_3D_FILE)
grid_file.save(OUTPUT_GRID_FILE)

***
# <font color=blue><center> SECTION-03: END </center></font>
***
<br>
<br>

<br> <br>

# <font color=blue><center> SECTION-04: Creating Load File (damask.Config object) </center></font>

## <font color=red><center> IMPORTANT NOTE: In the subsections only run general and your load case cells, leave others alone </center></font>
#### <font color=red> &emsp; &emsp; **NOTE**: load file is a must for damask simulation, it provides load conditions/steps applied on the material

#### <font color=red> Function Arguments Explanation </font>
1. ##### <font color=red> t : </font> Time for that part of loadstep to happen, eg. 
    1. ###### if ramp up happens in one second than t = 1
    2. ###### if elastic region is approximately 8 seconds then t = 8
    
2. ##### <font color=red> N : </font> Total discretization i.e. how much the time is divided into steps
    1. ###### each second will be divided into N/t steps
    2. ###### if N is large, steps are small, and if N is small steps are large (this may sometime lead to non-convergence)

3. ##### <font color=red> f_out: </font> Frequency of output, not every timestep output is saved this decides after how many times steps output needs to be saved
    1. ###### every f_out time_step output will be saved
    2. ###### if f_out = 8, then every 8th output will be saved

## <center><font color=purple> SECTION-04.1: Tensile Test Load File Creation </font></center>

### <font color=cadetblue> &emsp; &emsp; General Constants </font>

In [ ]:
SOLVER_NAME: str = "spectral_basic"
# defines the solver used to solve equations

LOADFILE_NAME: str = "Tension_Y.yaml"
# Your output load file name

### <font color=cadetblue> &emsp; &emsp; General Functions </font>

In [ ]:
def inversion(l,fill=0):
    return [inversion(i,fill) if isinstance(i,list) else\
            fill if i == 'x' else 'x' for i in l]

load_case = damask.Config(solver={"mechanical": SOLVER_NAME}, loadstep=[])
# Defining load case

### <font color=cadetblue> &emsp; &emsp; Constants For Tensile Tests </font>

In [ ]:
# Elastic Part
ELASTIC_TIME: float = 8.0
# Approximate time for elastic part in experimental data
ELASTIC_DISCRETIZATION: int = 160
# How many timesteps to divide elastic time
ELASTIC_OUTPUT_FREQUENCY: int = 8

# Plastic Part
PLASTIC_TIME: float = 54.0
# Approximate time for rest of simulation (after elastic)
PLASTIC_DISCRETIZATION: int = 540
# How many timesteps to divide plastic time
PLASTIC_OUTPUT_FREQUNCY: int = 8

# Strain Rates in principal direction, put value in loading direction, 11: x direction, 22: y direction, 33: z direction
# Keep the remaining as unknown i.e. x
STRAIN_RATE_11: float = "x"
STRAIN_RATE_22: float = 10 ** (-3)
STRAIN_RATE_33: float = "x"

### <font color=cadetblue> &emsp; &emsp; Functions for Tensile Tests </font>

In [ ]:
def tensile_test(t: float = 60, N: int = 600, f_out: int = 8):
    
    dot_F = [[STRAIN_RATE_11, 0, 0],
             [0, STRAIN_RATE_22, 0],
             [0, 0, STRAIN_RATE_33]]
    
    P_11 = 0 if STRAIN_RATE_11 == "x" else "x"
    P_22 = 0 if STRAIN_RATE_22 == "x" else "x"
    P_33 = 0 if STRAIN_RATE_33 == "x" else "x"
    
    P = [[P_11, 'x', 'x'],
         ['x', P_22, 'x'],
         ['x', 'x', P_33]]
    loadstep = {"boundary_conditions": {'mechanical': {'dot_F': dot_F,
                                                       'P': P}},
                                        'discretization': {'t': t, 'N': N}, 'f_out': f_out}
    load_case["loadstep"].append(loadstep)

### <font color=cadetblue> &emsp; &emsp; Creating Full Loadstep and Writing File </font>

In [ ]:
# Creating load case
tensile_test(ELASTIC_TIME, ELASTIC_DISCRETIZATION, ELASTIC_OUTPUT_FREQUENCY)
tensile_test(PLASTIC_TIME, PLASTIC_DISCRETIZATION, PLASTIC_OUTPUT_FREQUNCY)

# Saving Load File
load_case.save(LOADFILE_NAME)

## <center><font color=purple> SECTION-04.2: Dwell Fatigue Load File Creation </font></center>

### <font color=cadetblue> &emsp; &emsp; General Constants </font>

In [ ]:
SOLVER_NAME: str = "spectral_basic"
# defines the solver used to solve equations

LOADFILE_NAME: str = "load_dwell_unload.yaml"
# Your output load file name

### <font color=cadetblue> &emsp; &emsp; General Functions </font>

In [ ]:
def inversion(l,fill=0):
    return [inversion(i,fill) if isinstance(i,list) else\
            fill if i == 'x' else 'x' for i in l]

load_case = damask.Config(solver={"mechanical": SOLVER_NAME}, loadstep=[])
# Defining load case

### <font color=cadetblue> &emsp; &emsp; Constants for Dwell Fatigue Tests </font>

In [ ]:
YIELD_STRENGTH: float = 700 * (10 ** 6)
# Yield strength of material in Pascal

MAXIMUM_STRESS_to_YIELD_STRENGTH: float = 0.95
# Ratio of maximum stress to yield strength: maximum_stress/Yield_strength

R_RATIO: float = 0.1
# Ratio of minimum stress to maximum stress: min_stress/max_stress

DWELL_PERIOD: float = 120
# Dwell period in seconds
DWELL_DISCRETIZATION: int = 480
DWELL_OUTPUT_FREQUENCY: int = 240

RAMP_UP_TIME: float = 1.0
RAMP_UP_DISCRETIZATION: int = 20
RAMP_UP_OUTPUT_FREQUENCY: int = 10

RAMP_DOWN_TIME: float = 1.0
RAMP_DOWN_DISCRETIZATION: int = 20
RAMP_DOWN_OUTPUT_FREQUENCY: int = 10
    
CYCLE_COUNT: int = 100

# Principal loading direction (direction as in dream3d file, check in paraview with xdmf file.)
STRESS_11: bool = True
STRESS_22: bool = False
STRESS_33: bool = False

### <font color=cadetblue> &emsp; &emsp; Functions for Dwell Tests </font>

In [ ]:
def ramp_up(t: float = 1., N: int = 20, f_out: int = 10):
    """Ramp Up part of fatigue or dwell fatigue test. t: time to ramp up, N: total division of time,
        f_out: frequency of output."""
    
    stress_11 = MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_11 else 0
    stress_22 = MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_22 else 0
    stress_33 = MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_33 else 0
    
    P = [[ stress_11, 'x' , 'x'],
         [ 'x', stress_22 , 'x' ],
         [ 'x', 'x' , stress_33 ]]

    loadstep = {'boundary_conditions':{'mechanical':{'dot_F':inversion(P),
                                                     'P':P}},
                                       'discretization':{'t': t,'N': N},'f_out': f_out}
    load_case['loadstep'].append(loadstep)
    

def dwell(t: float = 120., N: int = 480, f_out: int = 240):
    
    dot_P_11 = 0 if STRESS_11 else "x"
    dot_P_22 = 0 if STRESS_22 else "x"
    dot_P_33 = 0 if STRESS_33 else "x"
    
    dot_P = [[dot_P_11 ,'x','x'],
             ['x', dot_P_22,'x'],
             ['x','x', dot_P_33]]

    loadstep = {'boundary_conditions':{'mechanical':{'dot_P':dot_P,
                                                     'dot_F':inversion(dot_P)}},
                                       'discretization':{'t': t,'N': N}, 'f_out': f_out}
    load_case['loadstep'].append(loadstep)
    

def ramp_down(t: float = 1., N: int = 20, f_out: int = 10):
    
    stress_11 = R_RATIO * MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_11 else 0
    stress_22 = R_RATIO * MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_22 else 0
    stress_33 = R_RATIO * MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_33 else 0
    
    P = [[ stress_11, 'x' , 'x'],
         [ 'x', stress_22 , 'x' ],
         [ 'x', 'x' , stress_33 ]]
    loadstep = {'boundary_conditions':{'mechanical':{'P':P,
                                                     'dot_F':inversion(P)}},
                                       'discretization':{'t': t,'N': N}, 'f_out': f_out}
    load_case['loadstep'].append(loadstep)

### <font color=cadetblue> &emsp; &emsp; Creating Full Loadstep and Writing File </font>

In [ ]:
# Creating load case
for _ in range(CYCLE_COUNT):
    ramp_up(RAMP_UP_TIME, RAMP_UP_DISCRETIZATION, RAMP_UP_OUTPUT_FREQUENCY)
    dwell(DWELL_PERIOD, DWELL_DISCRETIZATION, DWELL_OUTPUT_FREQUENCY)
    ramp_down(RAMP_DOWN_TIME, RAMP_DOWN_DISCRETIZATION, RAMP_DOWN_OUTPUT_FREQUENCY)

# Saving Load file
load_case.save(LOADFILE_NAME)

## <center><font color=purple> SECTION-04.3: Normal Fatigue Load File Creation </font></center>

### <font color=cadetblue> &emsp; &emsp; General Constants </font>

In [ ]:
SOLVER_NAME: str = "spectral_basic"
# defines the solver used to solve equations

LOADFILE_NAME: str = "load_unload.yaml"
# Your output load file name

### <font color=cadetblue> &emsp; &emsp; General Functions </font>

In [ ]:
def inversion(l,fill=0):
    return [inversion(i,fill) if isinstance(i,list) else\
            fill if i == 'x' else 'x' for i in l]

load_case = damask.Config(solver={"mechanical": SOLVER_NAME}, loadstep=[])
# Defining load case

### <font color=cadetblue> &emsp; &emsp; Constants for Fatigue Tests </font>

In [ ]:
YIELD_STRENGTH: float = 280 * (10 ** 6)
# Yield strength of material in Pascal

MAXIMUM_STRESS_to_YIELD_STRENGTH: float = 0.95
# Ratio of maximum stress to yield strength: maximum_stress/Yield_strength

R_RATIO: float = 0.1
# Ratio of minimum stress to maximum stress: min_stress/max_stress

RAMP_UP_TIME: float = 1.0
RAMP_UP_DISCRETIZATION: int = 20
RAMP_UP_OUTPUT_FREQUENCY: int = 10

RAMP_DOWN_TIME: float = 1.0
RAMP_DOWN_DISCRETIZATION: int = 20
RAMP_DOWN_OUTPUT_FREQUENCY: int = 10

CYCLE_COUNT: int = 100

### <font color=cadetblue> &emsp; &emsp; Functions for Normal Fatigue Tests </font>

In [ ]:
def ramp_up(t: float = 1., N: int = 20, f_out: int = 10):
    """Ramp Up part of fatigue or dwell fatigue test. t: time to ramp up, N: total division of time,
        f_out: frequency of output."""
    
    stress_11 = MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_11 else 0
    stress_22 = MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_22 else 0
    stress_33 = MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_33 else 0
    
    P = [[ stress_11, 'x' , 'x'],
         [ 'x', stress_22 , 'x' ],
         [ 'x', 'x' , stress_33 ]]

    loadstep = {'boundary_conditions':{'mechanical':{'dot_F':inversion(P),
                                                     'P':P}},
                                       'discretization':{'t': t,'N': N},'f_out': f_out}
    load_case['loadstep'].append(loadstep)
    

def ramp_down(t: float = 1., N: int = 20, f_out: int = 10):
    
    stress_11 = R_RATIO * MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_11 else 0
    stress_22 = R_RATIO * MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_22 else 0
    stress_33 = R_RATIO * MAXIMUM_STRESS_to_YIELD_STRENGTH * YIELD_STRENGTH if STRESS_33 else 0
    
    P = [[ stress_11, 'x' , 'x'],
         [ 'x', stress_22 , 'x' ],
         [ 'x', 'x' , stress_33 ]]
    loadstep = {'boundary_conditions':{'mechanical':{'P':P,
                                                     'dot_F':inversion(P)}},
                                       'discretization':{'t': t,'N': N}, 'f_out': f_out}
    load_case['loadstep'].append(loadstep)

### <font color=cadetblue> &emsp; &emsp; Creating Full Loadstep and Writing File </font>

In [ ]:
# Creating load case
for _ in range(CYCLE_COUNT):
    ramp_up(RAMP_UP_TIME, RAMP_UP_DISCRETIZATION, RAMP_UP_OUTPUT_FREQUENCY)
    ramp_down(RAMP_DOWN_TIME, RAMP_DOWN_DISCRETIZATION, RAMP_DOWN_OUTPUT_FREQUENCY)

# Saving Load file
load_case.save(LOADFILE_NAME)